In [1]:
import sys
sys.path.append('../../laddie/src/')

import numpy as np
import xarray as xr
import pandas as pd
import os
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo

## Steady geometry

In [3]:
# Select ice shelf
shelfname = 'PI'

# Read in BedMachine data
BM = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')

if shelfname == 'CD':
    x0,x1,y0,y1 = 3445,3740,7730,8070   # CD
elif shelfname == 'PI':
    x0,x1,y0,y1 = 3280,3560,7150,7400   # PI

# Cut out shelf region
shelf = BM.isel(x=slice(x0,x1),y=slice(y0,y1))

shelf.mask[:] = xr.where(shelf.mask==1,2,shelf.mask)                    # adjust mask, mark bare ground as grounded ice (but thickness is zero there)
shelf['draft'] = (shelf.surface-shelf.thickness).astype('float64')      # add draft
shelf = shelf.assign_coords(t=0)                                        # add time stamp


shelf.to_netcdf(f'BedMachine_singleshelves/{shelfname}.nc')

## Variable geometry -- using Paolo et al. 2022 thinning rates

In [ ]:
# Select ice shelf
shelfname = 'CD'

# Specify number of new geometries to compute from thinning rates Paolo
updates = 30

# Read in BedMachine data
BM = xr.open_dataset('../../data/BedMachineAntarctica_2020-07-15_v02.nc')

if shelfname == 'CD':
    x0,x1,y0,y1 = 3445,3740,7730,8070   # CD
    thinning_grounded = 2.1
    thinning_shelf_fraction = 0.0085 

elif shelfname == 'PI':
    x0,x1,y0,y1 = 3280,3560,7150,7400   # PI
    thinning_grounded = 1.8
    thinning_shelf_fraction = 0.0045 

# Cut out shelf region
shelf = BM.isel(x=slice(x0,x1),y=slice(y0,y1))

shelf.mask[:] = xr.where(shelf.mask==1,2,shelf.mask)                    # adjust mask, mark bare ground as grounded ice (but thickness is zero there)
shelf['draft'] = (shelf.surface-shelf.thickness).astype('float64')      # add draft

shelf_original = shelf.assign_coords(t=0)                               # add time stamp
shelf_original = shelf_original.assign(Hf = 0*shelf.bed)                # add maximum floating thickness
shelf_original = shelf_original.assign(th_change = 0*shelf.bed)         # add variable for thickness change

shelf_newgeoms = [shelf_original]                                       # list of shelf geometries

th = []

# loop over number of updates
for i in range(updates):
    shelf_n = copy.copy(shelf)

    shelf_n = shelf_n.assign(Hf = -(1+(1/9))*shelf_n.bed)

    thickness_change_shelf  = xr.where(shelf_n.mask==3, shelf_n['thickness'] * thinning_shelf_fraction, 0)
    thickness_change_ground = xr.where(shelf_n.mask==2, thinning_grounded, 0)
    thickness_change = thickness_change_shelf+thickness_change_ground

    th.append(thickness_change)
    
    shelf_n = shelf_n.assign(th_change = thickness_change)
    
    shelf_n['thickness'] = shelf_n['thickness'] - thickness_change

    shelf_n['mask'] = xr.where(np.logical_and(shelf_n.mask==2, shelf_n.Hf>shelf_n.thickness),3, shelf_n.mask)
    
    shelf_n['surface'] = xr.where(shelf_n['mask']==3, shelf_n['thickness']*0.1, shelf_n['surface']-thickness_change)

    shelf_n['draft'] = (shelf_n.surface-shelf_n.thickness).astype('float64')

    shelf = shelf_n

    shelf_n = shelf_n.assign_coords(t=i+1)

    shelf_newgeoms.append(shelf_n)


    i = i + 1


shelf_set = xr.concat(shelf_newgeoms, dim='t')
shelf_set = shelf_set.isel(y=slice(None, None, -1))

shelf_set.to_netcdf(f'Paolo-variablegeom/{shelfname}_paolo_{updates}yrs_new.nc')